In [1]:
# start google earth engine
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AdQt8qhimkrD0B1lgCK-UQyKJGDZ9dzaY3rkWYFbfdWB6B4djryN9eIr0ys

Successfully saved authorization token.


# Geometries

In [240]:
#Anchorage Bounds
geometry_bounds = ee.FeatureCollection(
    'projects/earthengine-legacy/assets/projects/sat-io/open-datasets/geoboundaries/HPSCGS-ADM2'
).filter(
    ee.Filter.eq('shapeName', 'Anchorage Municipality' )
)

# geometry_bounds = ee.Geometry.Polygon(
#         [[[-149.63471833692705, 61.37637997036421],
#           [-149.63467005716478, 61.37604845864766],
#           [-149.6341497086159, 61.37608186696437],
#           [-149.63434282766497, 61.37637226082931]]])

# var geometry = /* color: #d63000 */ee.Geometry.Polygon(
#         [[[172.4398642141001, 54.86458910133146],
#           [170.5062704641001, 50.72323561655372],
#           [-174.37654203589992, 49.767776804959325],
#           [-153.72224516089992, 55.11670678008646],
#           [-150.38240141089992, 58.206023230143174],
#           [-161.10505766089992, 58.206023230143174],
#           [-170.24568266089992, 58.02031323063899],
#           [-173.76130766089992, 54.4067763412549]]]),
#     geometry2 = /* color: #98ff00 */ee.Geometry.Polygon(
#         [[[-175.07966703589992, 61.38671890892917],
#           [-169.71833891089992, 57.267751873745894],
#           [-148.97615141089992, 57.73993024027015]]]);

#  Import building features from OSM data and map the area getting function over the FeatureCollection.
building_outlines = ee.FeatureCollection('users/edtrochim/OSM_building_AK')

# Calculate Annual Degree Days 
- degree_days_30y_norm_regression (feature collection)
- degree_days_30y_norm (feature collection)
- degree_days_10y_norm (feature collection)

In [241]:
# Import daily temperature dataset
era5 = ee.ImageCollection('ECMWF/ERA5/DAILY')

# Add inital and end dates for the required period
iniDate = ee.Date('1981-01-01')
endDate = ee.Date('2020-12-31') 

In [242]:
# Select the temperature of air at 2m (temperature_2m) and restrict data to Alaska
land_surface_temperature = era5.filterDate(iniDate, endDate).select('mean_2m_air_temperature').filter(
     #ee.Filter.bounds(geometry_bounds.geometry())
    ee.Filter.bounds(geometry_bounds)
 )

In [243]:
# Convert from K to C and add Freezing Days (FD) and Thawing Days (TD)                                 
def convert_replace(image):
    props = image.toDictionary(image.propertyNames())
    image_C = image.subtract(273.15)
    FD = image_C.where(image_C.gt(0), 0).rename('FD')
    TD = image_C.where(image_C.lt(0), 0).rename('TD')

    return ee.Image(image_C.addBands([FD, TD]).rename('temperature_2m', 'FD', 'TD').setMulti(props))


daily_temp_celsius = land_surface_temperature.map(convert_replace)

In [244]:
# Create list of years
years = daily_temp_celsius.aggregate_array('year').distinct()

def map_to_sequence_years(year):
    return daily_temp_celsius.filter(ee.Filter.eq('year', year)).reduce(ee.Reducer.sum()).set('year', year)
    
# Sum annual degree days
degree_days_annual =  ee.ImageCollection.fromImages(
    # Map function to sequence of years
    years.map(map_to_sequence_years)
)


In [245]:
# Create 30 year norm of temperature, freezing and thawing days
degree_days_30y_norm_regression = degree_days_annual.filter(ee.Filter.And(
    ee.Filter.gte('year',1981),ee.Filter.lte('year',1981+29))
    )\
    .select(['FD_sum','TD_sum'])\
    .reduce(ee.Reducer.mean()).reduceRegions(
        collection=building_outlines,
        reducer=ee.Reducer.mean(),
        scale=50,
)

In [246]:
task = ee.batch.Export.table.toDrive(degree_days_30y_norm_regression, folder='GEE_Output', description='Anchorage_Data_group1', fileFormat='CSV')


In [247]:
task.start()

In [252]:
task.status()

{'state': 'FAILED',
 'description': 'Anchorage_Data_group1',
 'creation_timestamp_ms': 1660063293505,
 'update_timestamp_ms': 1660063618149,
 'start_timestamp_ms': 1660063301227,
 'task_type': 'EXPORT_FEATURES',
 'attempt': 1,
 'error_message': 'User memory limit exceeded.',
 'id': 'XSI5MM5CFTLCNYTEDS6EO3OC',
 'name': 'projects/earthengine-legacy/operations/XSI5MM5CFTLCNYTEDS6EO3OC'}

In [209]:
# Create modern 10 year norm of temperature, freezing and thawing days
degree_days_10y_norm = degree_days_annual.filter(ee.Filter.And(
    ee.Filter.gte('year',2011),ee.Filter.lte('year',2011+9))
    )\
    .select(['FD_sum','TD_sum'])\
    .reduce(ee.Reducer.mean()).reduceRegions(
        collection=building_outlines,
        reducer=ee.Reducer.mean(),
        scale=50,
)

In [210]:
# Create modern 30 year norm of temperature, freezing and thawing days
degree_days_30y_norm = degree_days_annual.filter(ee.Filter.And(
    ee.Filter.gte('year',1991),ee.Filter.lte('year',1991+29))
    )\
    .select(['FD_sum','TD_sum'])\
    .reduce(ee.Reducer.mean()).reduceRegions(
        collection=building_outlines,
        reducer=ee.Reducer.mean(),
        scale=50,
)

# Building Area --> building_outlines (feature collection), building_area (feature collection)

In [211]:
#  This function computes a feature's geometry area and adds it as a property.
def addArea(feature):
    return feature.set({'areasq_ft': feature.geometry().area().multiply(10.7639)})

building_area = building_outlines.map(addArea).select(['areasq_ft']).filter(
    #ee.Filter.bounds(geometry_bounds.geometry())
    ee.Filter.bounds(geometry_bounds)
)


# Building Age --> building_age (feature collection)

In [212]:
# Import world settlements footprints data - for year of built
wsf = ee.ImageCollection("projects/sat-io/open-datasets/WSF/WSF_EVO").select('b1').filter(
    #ee.Filter.bounds(geometry_bounds.geometry())
    ee.Filter.bounds(geometry_bounds)
)

building_age = wsf.reduce(ee.Reducer.min()).reduceRegions(
        collection=building_outlines,
        reducer=ee.Reducer.mean(),
        scale=10,
)


In [214]:
# get buildings with age as None
building_null_age = combined_data.filter(ee.Filter.eq('age', None))

def buffer_poly_1000(feature):
    return feature.buffer(1000)

# calculate buffers around buildings with null age
buffered_buildings_1km = building_null_age.map(buffer_poly_1000)

# keep buildings that intersect with buffer buildings
def is_neighbor(polygon, inputGeom):
    polygon.intersection({'right': inputGeom})

building_not_null_age = combined_data.filter(
  ee.Filter.notNull(combined_data.propertyNames()))

def mean_neighbor(feature):
    return feature.reduceRegions(
    collection = building_not_null_age,
    reducer=ee.Reducer.mean(),
    scale=5
)
null_building_age_mean_neighbor = buffered_buildings_1km.map(mean_neighbor)

null_building_age_mean_neighbor.filter(ee.Filter.lt(2000)).set('age', 1985) # set the age = 1985

null_building_age_mean_neighbor.filter(ee.Filter.gte(2000)).set('age', 2015) # set the age = 2015

building_age = null_building_age_mean_neighbor.merge(building_not_null_age)

AttributeError: 'Feature' object has no attribute 'reduceRegions'

# Building Height --> building_height (array)

In [162]:
# Import FABDEM (Forest And Buildings removed Copernicus 30m DEM) - for elevation
fabdem = ee.ImageCollection("projects/sat-io/open-datasets/FABDEM").filter(
    #ee.Filter.bounds(geometry_bounds.geometry())
    ee.Filter.bounds(geometry_bounds)
)
ground_elevation = fabdem.mosaic()

# Import GLO--with buildings
glo30 = ee.ImageCollection("projects/sat-io/open-datasets/GLO-30")
building_elevation = glo30.mosaic().setDefaultProjection('EPSG:3857',None,30)

In [163]:
# Calculate the max height for each building
building_height = building_elevation.reduceRegions(collection= building_outlines,
  reducer=ee.Reducer.max(),
  scale=10,
)

In [164]:
# Calculate the 75th percentile height for each building
building_height_perc = building_elevation.reduceRegions(collection= building_outlines,
  reducer=ee.Reducer.percentile([75]),
  scale=10,
)

In [165]:
# add 5m buffers to each feature
def buffer_poly_5(feature):
    return feature.buffer(5)

buffered_buildings = building_outlines.map(buffer_poly_5)

In [166]:
# Calculate the min height for the buffered buildings
buffered_buildings_min_height = building_elevation.reduceRegions(collection=buffered_buildings,
  reducer= ee.Reducer.min(),
  scale=10
)

# Calculate difference between max of buildings and min of buffer + building
max_height_array = ee.Array(buffered_buildings_min_height.aggregate_array('max'))
min_height_array = ee.Array(buffered_buildings_min_height.aggregate_array('min'))

building_height = max_height_array.subtract(min_height_array)


# Combine everything and export

- degree_days_30y_norm_regression (feature collection)
- degree_days_10y_norm (feature collection)
- degree_days_30y_norm(feature collection)
- building_outlines (feature collection)
- building_area (feature collection)
- building_age (feature collection)
- building_height (array)

In [167]:
combined_data = degree_days_30y_norm_regression.merge([building_outlines, building_area, building_age, building_height, degree_days_10y_norm, degree_days_30y_norm])


In [168]:
type(combined_data)

ee.featurecollection.FeatureCollection

# Regression Relationship

In [201]:
# Regression Eqn
# y = -1.204x + 2514.6

def reg(feature):
    age  = ee.Number(feature.get('age'))
    sq_ft  = ee.Number(feature.get('sq_ft'))
    return feature.set({ 'heat_load': (age.multiply(-1.204).add(2514.6).multiply(sq_ft))})

# Apply regression relationship to every building that has a non null building age
building_not_null_age = combined_data.filter(
  ee.Filter.notNull(combined_data.propertyNames())) # unsure about this: combined_data.first().propertyNames()

building_null_age = combined_data.filter(ee.Filter.eq('age', None))


building_null_age.set('age', 1985)

btu_out = building_not_null_age.map(reg)


In [202]:
# Merging btu & null collections
combined_data_btu = btu_out.merge(building_null_age)

In [173]:
combined_data_final = combined_data.merge(combined_data_btu)

In [174]:
# ['degree_days_40y_norm', 'building_outlines', 'building_area', 'building_age', 'building_height']

In [175]:
# indexes = ee.List(combined_data_final.aggregate_array('system:index'))
# ids = ee.List.sequence(1, indexes.size())
# idByIndex = ee.Dictionary.fromLists(indexes, ids)

# def add_ID(feature):
#     return feature.set('id', idByIndex.get(feature.get('system:index')))

# datasetWithId = combined_data_final.map(add_ID)

In [176]:
# group1 = combined_data_final.filter(ee.Filter.lte('ids', 500))

In [177]:
# group1.select['ids']

In [215]:
task = ee.batch.Export.table.toDrive(building_height, folder='GEE_Output', description='Anchorage_Data_group1', fileFormat='CSV', selectors=['building_height'])


In [216]:
task.start()

In [239]:
task.status()

{'state': 'FAILED',
 'description': 'Anchorage_Data_group1',
 'creation_timestamp_ms': 1660062147939,
 'update_timestamp_ms': 1660062778319,
 'start_timestamp_ms': 1660062162095,
 'task_type': 'EXPORT_FEATURES',
 'attempt': 1,
 'error_message': 'Array.subtract: Binary array lengths should be equal, but found: axis 0 has lengths 138920 and 141088.',
 'id': 'BYIEQAMDMVNHZCIYZXFRIHVG',
 'name': 'projects/earthengine-legacy/operations/BYIEQAMDMVNHZCIYZXFRIHVG'}

In [145]:
# # Export to CSV :)
# labels = ee.List(['40y avg degree days', '.geo','sq feet', 'age', 'meters', 'heat_load'])

# Export.table.toDrive({
#   collection: combined_data_final,
#   description: 'Anchorage_Data',
#   folder: 'GEE_Output',
#   fileFormat: 'CSV',
#   selectors: labels.getInfo()
# })

In [ ]:
import geemap

In [ ]:
df = geemap.ee_to_pandas(combined_data_final)
df.head()